In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_numerics
import ariel_gp
import ariel_load
import time
import numpy as np
import multiprocess
import importlib
import copy
import cupy as cp
import matplotlib.pyplot as plt
from tqdm import tqdm

local


In [2]:
train_data=kgs.load_all_train_data()

In [3]:
importlib.reload(ariel_numerics)
loaders = ariel_load.default_loaders()
for ii in range(2):
    loaders[ii].apply_pixel_corrections.mask_hot= False
res = []
plt.figure()
for d in tqdm(train_data[:20]):
    d = copy.deepcopy(d)
    d.load_to_step(3, loaders)
    residuals = ariel_numerics.remove_trend_cp(d.transits[0].data[0].data)[10:-10,12:20,12:20]
    
    assert not cp.any(cp.isnan(residuals))
    
    # d.load_to_step(5, loaders)
    # x=cp.mean(d.transits[0].data[1].data,0)
    # x=x/kgs.rms(x)
    # if d.planet_id == train_data[0].planet_id:
    #     x0 = copy.deepcopy(x)
    # else:
    #     plt.plot((x/x0).get())
    
    res.append(residuals)
        
    d.load_to_step(0, ariel_load.default_loaders())
    
res = cp.concatenate(res)

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:28<00:00,  1.42s/it]


<Figure size 640x480 with 0 Axes>

In [4]:
res.shape

(26600, 8, 8)

In [5]:
pca.

SyntaxError: invalid syntax (3275565107.py, line 1)

In [ ]:
N=15
from sklearn.decomposition import PCA
pca = PCA(n_components=N)
dat = res.reshape(-1,64).get()
pca.fit(dat)
components = pca.components_          # Principal directions (eigenvectors)
components.shape
#explained_variance = 1-np.cumsum(pca.explained_variance_ratio_)  # Variance explained by each PC
plt.figure()
plt.semilogy(pca.explained_variance_)
for ii in range(N):
    #plt.figure()
    #plt.imshow(components[ii,:].reshape(8,8))
    
    scores = pca.transform(dat)[:, :ii]             # (n_samples, ii)
    recon = scores @ pca.components_[:ii, :]      # (n_samples, 64)
    recon += pca.mean_                            # add back mean

    # Residual
    residual = dat - recon                          # (n_samples, 64)

    # Example: visualize ith PC
    #plt.figure()
    #plt.imshow(components[ii-1, :].reshape(8, 8))

    # Example: track residual norm
    #res_norm = np.linalg.norm(residual) / np.linalg.norm(dat)
    #print(f"Residual ratio after {ii} components: {res_norm:.3f}")
    
    cov = (residual.T@residual)/residual.shape[0]
    plt.figure()
    plt.imshow(cov)
    plt.colorbar()
    plt.title(ii)
    
    
    # data = res.reshape(-1,64).get9)
    # data = data - components[:ii+1,:]

In [ ]:
components = pca.components_          # Principal directions (eigenvectors)
components.shape
explained_variance = 1-np.cumsum(pca.explained_variance_ratio_)  # Variance explained by each PC
plt.figure()
plt.plot(explained_variance)

In [ ]:
import ariel_diagnostics
ariel_diagnostics.animate_3d_matrix(res[10:20,...].get())

In [ ]:
np.prod(residuals.shape)*8/1e9

In [ ]:
# cov = (resF.T@resF)
# plt.figure()
# plt.imshow(cov[490:500,490:500].get())
# plt.colorbar()

In [ ]:
15*32+15

In [ ]:
S = [];components = []; weights = []
dat = copy.deepcopy(res.reshape(res.shape[0],-1))
for ii in tqdm(range(3)):
    resF = dat#.reshape(-1,32,32)[:,14:18,14:18].reshape(-1,16)
    cov = (resF.T@resF)/resF.shape[0]
    plt.figure()
    plt.imshow(cov.get())
    plt.colorbar()
    plt.title(ii)
    this_weights, this_components, this_S = ariel_numerics.nan_pca(dat, 1, tol=1e-5, verbose=False)
    dat -= this_weights * this_components
    S.append(this_S.get())
    components.append(this_components)
    weights.append(this_weights)
    resF = dat
    cov = (resF.T@resF)/resF.shape[0]
plt.figure()
plt.semilogy(S)

In [ ]:
for ii in range(len(components)):
    plt.figure()
    plt.imshow(components[ii].reshape(32,32).get(), aspect='auto', interpolation='none')
    plt.colorbar()
    plt.title(ii)

In [ ]:
plt.plot(weights[0].get())
plt.xlim([0,100])